# **Maestría en Inteligencia Artificial Aplicada**

## **Curso: Proyecto Integrador**

### Tecnológico de Monterrey

### Prof Dra. Grettel Barceló Alonso y Dr. Luis Eduardo Falcón Morales

## Avance II de Proyecto

## Ingeniería de Características

## Integrantes del Equipo:
### - Erika Cardona Rojas            A01749170
### - Miriam Bönsch                  A01330346
### - Mardonio Manuel Román Ramírez  A01795265

In [ ]:
# Librerias
import pandas as pd
import yaml
import numpy as np
import statsmodels.formula.api as smf
from statsmodels.stats.multitest import multipletests

# Cargando Yaml
with open("../config.yaml", "r", encoding="utf-8") as file:
    config = yaml.safe_load(file)

import warnings
# Ignora solo los avisos de funciones que van a cambiar en el futuro
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

# Importando modulos
import sys
from pathlib import Path
# ----------------- Esto solo es para importar src de un folder antes
parent_folder = str(Path.cwd().parent)
if parent_folder not in sys.path:
    sys.path.append(parent_folder)

from src import functions as f
from src import modelsF as mf
from src import filtering_vars as fv

> # Generación de Nuevas Características

En esta etapa, realizaremos la estructuración y generación de nuevas variables basadas en el diseño experimental del estudio. El proceso se divide en dos ejes fundamentales:

- **Segmentación Temporal y de Grupo:** Identificación de los periodos de Intervención y Control para cada individuo, asignando etiquetas cronológicas (Pre y Post) a cada medición.
- **Derivación de Variables:** Con el fin de capturar la dinámica de respuesta al tratamiento, generaremos métricas de interacción. Esto incluye el cálculo de deltas ($\Delta$), tasas de cambio y variables booleanas (flags) de incremento, permitiendo una interpretación más profunda del efecto de los polifenoles en el rendimiento cognitivo.

In [ ]:
# Cargando Base de Datos
df = pd.read_excel(r"../Data/Final_DF.xlsx")

In [ ]:
# Creando columnas identificadoras
df['Treatment'] = np.where(df['tratamiento'] == 1 , 'Intervencion', 'Control')
df['Time'] = np.where(df['tiempo'] == 1 , 'Pre', 'Post')

# Eliminando otras variables identificadoras
df = df.drop(['visita', 'periodo', 'tiempo', 'tratamiento'], axis='columns')

# Transformación Yeo–Johnson

En esta fase de ingeniería de características se generó una versión transformada del conjunto de datos utilizando la transformación de potencia Yeo–Johnson, aplicada a las variables numéricas.

El dataset incluía múltiples tipos de mediciones biomédicas y conductuales, muchas de las cuales presentan distribuciones altamente asimétricas:

* biomarcadores con colas largas
* variables de consumo con muchos ceros
* mediciones fisiológicas con valores extremos naturales

En estos casos, trabajar con los valores originales puede generar:

* alta influencia de outliers
* varianza inestable
* dificultad para aplicar modelos estadísticos interpretables

Por ello, la transformación busca mejorar la estructura estadística del dataset antes del modelado.

# Reducción De Dimensionalidad

No se realizará ninguna de estas técnicas, la justificación será la siguiente:

## **PCA**
Técnicamente: Realiza una transformación lineal ortogonal para proyectar los datos en un nuevo espacio donde la varianza se maximiza. Las nuevas características (PC1, PC2...) son combinaciones lineales de todas las variables originales.

Por qué no: El "Componente 1" podría ser 0.5*Edad - 0.2*Glucosa + 0.1*Peso. Clínicamente, esto es casi imposible de traducir en un perfil de paciente accionable.

> # Tecnicas De Filtrado

# Umbral De Varianza

El algoritmo implementado (`variance_filter`) es una técnica de **selección de características no supervisada**. Su objetivo es identificar y descartar variables que aportan poca o nula información al modelo (baja varianza) o que tienen una distribución extremadamente desbalanceada.

La estrategia se adapta dinámicamente según el tipo de variable (Numérica, Ordinal, Binaria o Categórica).

### 1. Preprocesamiento: Transformación Z-Score
Para las variables numéricas, el código utiliza una función auxiliar `zscore_df`.
* **Lógica:** $Z = \frac{x - \mu}{\sigma}$
* **Objetivo:** Estandarizar las variables para que tengan una media de 0 y una desviación estándar de 1.
* **Importancia:** Esto elimina la escala de las variables originales. Sin esto, una variable con unidades pequeñas (ej. 0.0001) podría parecer que tiene "baja varianza" comparada con una de unidades grandes (ej. 1000), aunque ambas sean informativas.

### 2. Reglas de Filtrado por Tipo de Variable

A continuación se describe el criterio de descarte para cada tipo de dato:

#### A. Variables Numéricas (`Numerica`)
Se busca eliminar variables **casi-constantes**.
1.  **Transformación:** Se aplica el `zscore_df` a la columna.
2.  **Cálculo:** Se calcula la varianza de los valores Z (normalizados).
3.  **Criterio:**
    * Se conserva si: `Varianza(Z) >= continuous_var_threshold` (Default: 0.01).
    * *Interpretación:* Si la varianza de la variable normalizada es cercana a 0, significa que la variable original es prácticamente un valor único repetido.

#### B. Variables Ordinales (`Ordinal`)
Se asume que una variable ordinal debe tener suficiente granularidad para establecer un orden significativo.
1.  **Cálculo:** Se cuentan los valores únicos (`nunique`).
2.  **Criterio:**
    * Se conserva si: `nunique >= ordinal_min_unique` (Default: 3).
    * *Interpretación:* Si una variable ordinal tiene menos de 3 niveles, se considera que tiene poca capacidad para diferenciar rangos y podría tratarse mejor como binaria o descartarse.

#### C. Variables Binarias (`Binaria`)
Se busca eliminar variables con **baja varianza debido a desbalance extremo**.
1.  **Cálculo:** Se obtiene la frecuencia relativa de la clase más común (`top_freq`).
2.  **Derivación:** Se calcula la frecuencia de la clase minoritaria: $Minoría = 1.0 - top\_freq$.
3.  **Criterio:**
    * Se conserva si: `Minoría >= binary_minority_freq` (Default: 0.05 o 5%).
    * *Interpretación:* Si el 95% o más de los datos pertenecen a una sola clase (ej. "Sí"), la variable aporta muy poca información discriminativa y puede sesgar el modelo.

#### D. Variables Categóricas (`Categorica` / Nominales)
Se busca eliminar variables constantes (un solo valor).
1.  **Cálculo:** Se cuentan los valores únicos.
2.  **Criterio:**
    * Se conserva si: `nunique >= 2`.
    * *Interpretación:* Si solo existe 1 categoría para todas las filas, la variable es una constante y no sirve para predecir nada.

### Resumen de Parámetros

| Parámetro | Valor Default | Descripción |
| :--- | :--- | :--- |
| `continuous_var_threshold` | `0.01` | Umbral mínimo de varianza para numéricas (post-zscore). |
| `binary_minority_freq` | `0.05` | Porcentaje mínimo que debe representar la clase minoritaria. |
| `ordinal_min_unique` | `3` | Cantidad mínima de niveles para considerar una variable ordinal válida. |

In [5]:
# Seleccionando columnas para filtrado, excluyendo las identificadoras
uv_cols = list(set(df.columns) - set(['id','grupo','Treatment','Time']))

# Infiriendo tipo de variables
type_cols = fv.infer_var_types(df,uv_cols)


In [7]:
# Aplicando Análisis de Umbral de varianza
variables_ok, df_reporte_var = fv.variance_filter(df,uv_cols,type_cols)

print(f"Se tienen {df_reporte_var[df_reporte_var['keep'] == False].shape[0]} variables candidatas a eliminar.")
df_reporte_var.to_excel('../Entregables/UdeBarcelona/umbral_varianza.xlsx', index= False)

Se tienen 13 variables candidatas a eliminar.


PermissionError: [Errno 13] Permission denied: '../Entregables/UdeBarcelona/umbral_varianza.xlsx'

### Debido a que las decisiones de eliminar variables se deben de discutir con los expertos, se discutirá el reporte con ellos.
### Se discutirán las 13 variables con los expertos.
---

# Correlaciones
# Técnica de Selección de Características: Filtro de Redundancia Basado en Relevancia (BDNF)

algoritmo de selección de variables diseñado para optimizar el conjunto de predictores en el estudio de polifenoles y BDNF.

## 1. Descripción de la Técnica
La función `correlation_filter_bdnf` actúa como un filtro inteligente que reduce la dimensionalidad del dataset bajo dos principios:

1.  **Eliminación de Redundancia (Multicolinealidad):** Detecta pares de variables con una correlación de Spearman $\geq 0.90$. Una correlación tan alta indica que ambas variables aportan prácticamente la misma información.
2.  **Criterio de Relevancia Biológica:** En lugar de eliminar una variable al azar, el algoritmo calcula la correlación de cada una contra el **BDNF** (Variable Objetivo). Se conserva aquella que tenga el coeficiente de correlación más alto, asegurando que el modelo final mantenga los predictores con mayor potencial explicativo.
3.  **Calidad de Datos:** Si no existe una diferencia clara en la relevancia, se selecciona la variable con menos valores faltantes (*NaNs*).

---

## 2. Justificación: ¿Por qué aplicar el filtro en el tiempo "Pre"?

El diseño del experimento cuenta con 4 registros por individuo (Tratamiento/Control en tiempos Pre/Post). Es metodológicamente crucial que la selección de variables se realice **exclusivamente con los datos del tiempo "Pre"**.

### A. Caracterización de la Línea Base
En el estado **Pre**, las variables reflejan la fisiología basal y la homeostasis natural del individuo. Las correlaciones aquí presentes son "puras", ya que no han sido alteradas por la intervención de polifenoles o el efecto del tiempo en el estudio.

### B. Consistencia en Medidas Repetidas
Para comparar el efecto de la intervención, debemos medir exactamente las mismas variables en el tiempo Pre y en el Post. Seleccionar las "mejores" variables en el Pre garantiza que evaluaremos el cambio ($\Delta$) sobre una estructura biológica robusta y predefinida.

---

## 3. Resumen de Reglas de Decisión

| Criterio | Condición | Acción |
| :--- | :--- | :--- |
| **Filtro de Redundancia** | $r \geq 0.90$ | Identificar par de variables redundantes. |
| **Prioridad por Target** | $r(A, BDNF) > r(B, BDNF)$ | Mantener **A**, eliminar **B**. |
| **Punto de Corte** | Tiempo == "Pre" | Filtrar dataset para todo el análisis de correlaciones. |

In [19]:
variables_ok_cor, df_reporte_var_cor = fv.correlation_filter_bdnf(df[df['Time'] == 'Pre'],uv_cols,type_cols)

print(f"Se tienen {df_reporte_var_cor.shape[0]} variables candidatas a eliminar.")
df_reporte_var_cor.to_excel('../Entregables/UdeBarcelona/correlaciones.xlsx', index= False)

Se tienen 49 variables candidatas a eliminar.


### Debido a que las decisiones de eliminar variables se deben de discutir con los expertos, se discutirá el reporte con ellos.
### Se discutirán las 49 variables con los expertos.
---

---
# Análisis: Modelo Lineal Mixto (LMM) y Difference-in-Differences (DiD)
Este análisis utiliza un **Modelo Lineal Mixto (Mixed Linear Model - LMM)**. Dado el diseño del ensayo clínico (mediciones repetidas en los mismos sujetos), este enfoque es superior a un ANOVA simple o pruebas t, ya que modela explícitamente la correlación intra-sujeto. El propósito principal de este análisis será identificar variables con mucha importancia con un modelo relativamente sencillo, si bien no decidiremos eliminar ninguna variable con este análisis, si podremos decidir cuáles tenemos que conservar.

## Fundamento Teórico: Diferencia en Diferencias (DiD)

El objetivo central del estudio no es solo determinar si un grupo es mejor que otro en promedio, ni si los pacientes mejoran simplemente por el paso del tiempo. El objetivo es determinar si la **tasa de mejora** es distinta entre los grupos.

## Diseño
- N = 42 sujetos (ID)
- Entre-sujetos: `Group` (1 vs 2)
- Intra-sujeto (repetidas): 
  - `Treatment` (Intervencion vs Control)
  - `Time` (Pre vs Post)
- Cada sujeto aporta 4 observaciones: (Treatment × Time)

Este diseño induce correlación intra-sujeto que invalida pruebas independientes simples.

## Modelo
Para cada variable numérica Y:

Y ~ Group * Treatment * Time + (1 | ID)

Donde:
- `Group`, `Treatment`, `Time` son efectos fijos
- `(1|ID)` es intercepto aleatorio (captura heterogeneidad basal por sujeto)

## Interpretación DiD
- La interacción `Treatment:Time` representa el efecto Difference-in-Differences (cambio pre→post en Intervention menos cambio pre→post en Control) para el grupo de referencia.
- La interacción triple `Group:Treatment:Time` evalúa si el efecto DiD difiere entre Group 1 y Group 2.

Conceptualmente, buscamos la interacción estadística conocida como "Difference-in-Differences":

$$\text{Efecto Neto} = (\Delta_{\text{Grupo Experimental}}) - (\Delta_{\text{Grupo Control}})$$

$$\text{Efecto} = (\bar{X}_{A, \text{Post}} - \bar{X}_{A, \text{Pre}}) - (\bar{X}_{B, \text{Post}} - \bar{X}_{B, \text{Pre}})$$

## Salidas reportadas
- Coeficientes (betas), errores estándar, IC95%, p-values
- Ajuste por comparaciones múltiples (FDR) a través de variables (200+)
- Efectos marginales (contrastes) para estimar DiD por grupo

## Ventajas
- Modela explícitamente la estructura de medidas repetidas
- Usa toda la información sin colapsar datos a deltas
- Es más robusto que t-tests/ANOVA simple en presencia de correlación intra-sujeto

In [ ]:
# Preparando el DataFrame
# Excluiremos las variables deltas, por la naturaleza de esas mediciones.
working_df, numeric_vars = f.prepare_dataframe(df, exclude_cols= config['Tratamiento_Variables']['Objetivos'][1:])

working_vars = ['id','grupo','Treatment','Time'] + numeric_vars

working_df = working_df[working_vars]
print(f"Realizaremos este primer análisis utilizando {len(numeric_vars) - 1} variables numéricas.")

>> ## Ajuste de un LMM por variable

In [ ]:
results, failed = mf.run_lmm_screen(
    working_df,
    id_col="id",
    group_col="grupo",
    treatment_col="Treatment",
    time_col="Time"
)

### Interpretando hallazgos

In [ ]:
results.columns

In [ ]:
# Interpretando
df_interpretado = f.interpretar_hallazgos_final(results)

# Filtrar solo lo relevante para mostrar
cols_reporte = [
    "variable",                # Biomarcador
    "Interpretacion",          # Tu conclusión verbal
    
    # Magnitud del Efecto Clínico (Estimaciones Netas)
    "Efecto_Suplemento_G1",    # ¿Cuánto mejoró el Grupo 1?
    "Efecto_Suplemento_G2",    # ¿Cuánto mejoró el Grupo 2?
    
    # Desglose Estadístico (Evidencia)
    "Triple_q_FDR",            # ¿Es diferente entre grupos? (q-value ajustado)
    "DiD_q_FDR",               # ¿Es efecto del tratamiento? (q-value ajustado)
    
    # Componentes Crudos (Para referencia técnica si te preguntan)
    "Triple_beta",             # El diferencial puro
    "Variacion_Natural"        # El efecto tiempo/aprendizaje (Time_beta)
]

# Mostrar solo las variables donde hubo ALGÚN hallazgo (Prioridad 1 o 2)
hallazgos_significativos = df_interpretado[df_interpretado["Ranking"] < 3][cols_reporte]

print(f"Se encontraron {len(hallazgos_significativos)} biomarcadores con respuesta significativa.")

In [ ]:
hallazgos_significativos.to_excel('../Entregables/hallazgos_LMM.xlsx', index= False)